In [89]:
import glob
import shutil
import numpy as np
import pandas as pd
from biom import load_table
from skbio.stats.composition import clr_inv as softmax
from skbio.util import get_data_path
from scipy.stats import spearmanr, pearsonr
from scipy.sparse import coo_matrix
from scipy.spatial.distance import pdist
from mmvec.multimodal import MMvec
from mmvec.util import random_multimodal
from tensorflow import set_random_seed
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

%matplotlib inline

In [90]:
microbe_total = 1000
metabolite_total = 10000
np.random.seed(1)
res = random_multimodal(
    num_microbes=100, num_metabolites=100, num_samples=500,
    latent_dim=3, 
    microbe_total=microbe_total, metabolite_total=metabolite_total, seed=1
)
(microbes, metabolites, X, B,
 U, Ubias, V, Vbias) = res
num_train = 10
trainX = microbes.iloc[:-num_train]
testX = microbes.iloc[-num_train:]
trainY = metabolites.iloc[:-num_train]
testY = metabolites.iloc[-num_train:]

In [91]:
def mmvec_fit(trainX, trainY, testX, testY):
    device_name='/device:GPU:0'
    np.random.seed(1)
    tf.reset_default_graph()
    n, d1 = trainX.shape
    n, d2 = trainY.shape
    with tf.Graph().as_default(), tf.Session() as session:
        set_random_seed(0)
        model = MMvec(beta_1=0.8, beta_2=0.9, latent_dim=3, 
                      device_name=device_name, batch_size=10000, 
                      learning_rate=1e-1)
        model(session,
              coo_matrix(trainX.values), trainY.values,
              coo_matrix(testX.values), testY.values)
        loss, cv = model.fit(epoch=500)

        res = softmax(model.ranks())
        return res

In [92]:
from skbio.stats.composition import clr

def pearson_fit(microbes, metabolites):
    C = np.corrcoef(microbes.values.T, metabolites.values.T) 
    Cclr = np.corrcoef(clr(microbes.values+1).T, clr(metabolites.values+1).T)
    return C, Cclr

In [97]:
def black_box_function(uB, sigmaB, sigmaQ, uU, sigmaU, uV, sigmaV):
    microbe_total = 1000
    metabolite_total = 100000
    np.random.seed(1)
    try:
        res = random_multimodal(
            num_microbes=100, num_metabolites=100, num_samples=500,
            latent_dim=3, sigmaQ=sigmaQ, uU=uU, uV=uV, sigmaU=sigmaU, sigmaV=sigmaV,
            microbe_total=microbe_total, metabolite_total=metabolite_total, seed=1
        )
    except:
        return -1 # simulation broke
    (microbes, metabolites, X, B,
     U, Ubias, V, Vbias) = res
    num_train = 10
    trainX = microbes.iloc[:-num_train]
    testX = microbes.iloc[-num_train:]
    trainY = metabolites.iloc[:-num_train]
    testY = metabolites.iloc[-num_train:]

    res = mmvec_fit(trainX, trainY, testX, testY)
    C, Cclr = pearson_fit(trainX, trainY)
    U_ = np.hstack(
        (np.ones((U.shape[0], 1)), Ubias, U))
    V_ = np.vstack(
        (Vbias, np.ones((1, V.shape[1])), V))
    exp = softmax(np.hstack((np.zeros((microbes.shape[1], 1)), U_ @ V_)))
    mmvec_r, mmvec_p = spearmanr(np.ravel(res), np.ravel(exp))
    C_r, C_p = spearmanr(np.ravel(C[:100, 100:]), np.ravel(exp))
    Cclr_r, Cclr_p = spearmanr(np.ravel(Cclr[:100, 100:]), np.ravel(exp))
    print(mmvec_r, Cclr_r, C_r)
    sparsity = (microbes == 0).sum().sum() / (microbes.shape[0] * microbes.shape[1])
    c = max(np.nan_to_num(C_r), np.nan_to_num(Cclr_r))
    return (mmvec_r - c) * sparsity

In [98]:
from bayes_opt import BayesianOptimization

pbounds = {
    'uU': (-5, 5), 
    'uV': (-5, 5),
    'uB': (-5, 5),
    'sigmaU': (1e-1, 10), 
    'sigmaV': (1e-1, 10),
    'sigmaB': (1e-1, 10),
    'sigmaQ': (1e-1, 10)
}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=0,
)
optimizer.maximize(
    init_points=2,
    n_iter=100,
)

|   iter    |  target   |  sigmaB   |  sigmaQ   |  sigmaU   |  sigmaV   |    uB     |    uU     |    uV     |
-------------------------------------------------------------------------------------------------------------


100%|██████████| 1108/1108 [00:05<00:00, 214.62it/s]


0.11618168190468721 -0.10824002934054129 nan
|  1        |  0.06365  |  5.533    |  7.18     |  6.067    |  5.494    | -0.7635   |  1.459    | -0.6241   |


100%|██████████| 884/884 [00:04<00:00, 214.57it/s]


0.15266481490537712 -0.09681824662797403 nan
|  2        |  0.09761  |  8.929    |  9.64     |  3.896    |  7.938    |  0.2889   |  0.6804   |  4.256    |


...

0.9987139659031397 0.01423857591438576 nan
|  100      |  0.6401   |  4.324    |  10.0     |  0.1      |  0.1      |  5.0      |  1.896    |  5.0      |


100%|██████████| 850/850 [00:03<00:00, 220.89it/s]


0.2227013061550131 -0.11254556870890944 nan
|  101      |  0.1455   |  0.1      |  10.0     |  0.1      |  0.1      |  5.0      |  0.8968   | -5.0      |


100%|██████████| 859/859 [00:03<00:00, 217.14it/s]


0.9996154521721545 0.002273426374734264 nan
|  102      |  0.6483   |  10.0     |  10.0     |  0.1      |  0.1      |  1.442    |  5.0      |  5.0      |


In [99]:
optimizer.max

{'target': 0.6483976936510781,
 'params': {'sigmaB': 0.1,
  'sigmaQ': 10.0,
  'sigmaU': 0.1,
  'sigmaV': 0.1,
  'uB': 0.9299444487730857,
  'uU': 5.0,
  'uV': 5.0}}